In [2]:
#pip install datasets

In [3]:
#pip install accelerate -U

In [4]:
#pip install wandb

In [5]:
#pip install transformers

In [1]:

# Finetuning a language model

#<!--- @wandbcode{dlai_05} -->

#Let's see how to finetune a language model to generate character backstories using HuggingFace Trainer with wandb integration. We'll use a tiny language model (`TinyStories-33M`) due to resource constraints, but the lessons you learn here should be applicable to large models too!

#pip install datasets

#pip install wandb 3f1ee59462df44796372f2c8717640d164cb3878

from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from transformers import Trainer,TrainingArguments
import transformers
transformers.set_seed(42)
import wandb

wandb.login(anonymous="allow",relogin=True)

model_checkpoint = "roneneldan/TinyStories-33M"



wandb: (1) Private W&B dashboard, no account required
wandb: (2) Use an existing W&B account


wandb: Enter your choice: 1


wandb: You chose 'Private W&B dashboard, no account required'
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\siddhanna.janai/.netrc


In [2]:
# Sample DataFrame
# Load the Excel file
import pandas as pd
excel_file_path = 'mahabharata.xlsx'
data = pd.read_excel(excel_file_path)

df = pd.DataFrame(data)     

In [59]:
# As this dataset has no validation split, we will create one
#ds = ds.train_test_split(test_size=0.2, seed=42)

In [1]:
#df

In [4]:
from sklearn.model_selection import train_test_split
# Split the DataFrame into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)  # You can adjust the test_size and random_state as needed

print("Train DataFrame:")
print(train)

print("Test DataFrame:")
print(test)

Train DataFrame:
                                                 text  \
55  Character Name: Shalya's wives Character Race:...   
88  Character Name: Raja Janamejaya Character Race...   
26  Character Name: Ulupi Character Race: Naga Cha...   
42  Character Name: Ambalika Character Race: Human...   
69  Character Name: Yama Race: Deva Class: God of ...   
..                                                ...   
60  Character Name: Sanjaya Race: Human Class: Mes...   
71  Character Name: Kauravas (sons of Dhritarashtr...   
14  Character Name: Amba, Ambika, and Ambalika Cha...   
92  Character Name: Uttara Character Race: Human C...   
51  Character Name: Chitrasena's sons Character Ra...   

                                               target  
55  Shalya had many wives, all of whom were human ...  
88  Raja Janamejaya was the king of Hastinapura af...  
26  Ulupi was a Naga princess who was the wife of ...  
42  Ambalika was one of the three wives of Dhritar...  
69  Yama was the g

In [62]:
train = train.reset_index(drop=True)
#train

,text,target
0,Character Name: Shalya's wives Character Race:...,"Shalya had many wives, all of whom were human ..."
1,Character Name: Raja Janamejaya Character Race...,Raja Janamejaya was the king of Hastinapura af...
2,Character Name: Ulupi Character Race: Naga Cha...,Ulupi was a Naga princess who was the wife of ...
3,Character Name: Ambalika Character Race: Human...,Ambalika was one of the three wives of Dhritar...
4,Character Name: Yama Race: Deva Class: God of ...,Yama was the god of death. He was a powerful g...
...,...,...
75,Character Name: Sanjaya Race: Human Class: Mes...,Sanjaya was a blind sage who was the chariotee...
76,Character Name: Kauravas (sons of Dhritarashtr...,The Kauravas were the hundred sons of Dhritara...
77,"Character Name: Amba, Ambika, and Ambalika Cha...","Amba, Ambika, and Ambalika were the three prin..."
78,Character Name: Uttara Character Race: Human C...,Uttara was a princess and the wife of Abhimanyu.


In [63]:
test = test.reset_index(drop=True)
#test

,text,target
0,Character Name: Nidra Character Race: Personif...,Nidra was the goddess of sleep. She was known ...
1,Character Name: Nakula's wives Character Race:...,"Nakula had two wives, both of whom were celest..."
2,Character Name: Upapandavas (sons of Draupadi)...,The Upapandavas were the five sons of Draupadi...
3,Character Name: Sudarshana Character Race: Hum...,Sudarshana was a warrior who fought on the sid...
4,Character Name: Iravan Character Race: Naga Ch...,"Iravan was the son of Arjuna and Ulupi, a Naga..."
5,Character Name: Krishna Character Race: Yadava...,Krishna was an avatar of the god Vishnu. He wa...
6,Character Name: Vidura Character Race: Human C...,Vidura was the half-brother of Dhritarashtra a...
7,Character Name: Maharishi Veda Vyasa Character...,Maharishi Veda Vyasa was the author of the Mah...
8,Character Name: Bhishma Character Race: Hum...,"Bhishma, the eldest of the Pandava and Kaurava..."
9,Character Name: Arjuna Character Race: Huma...,"Arjuna, a skilled archer and prince of the Kur..."


#As this dataset has no validation split, we will create one
#ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [64]:
from transformers import AutoTokenizer
# Create a tokenizer from the model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token



loading file vocab.json from cache at C:\Users\siddhanna.janai/.cache\huggingface\hub\models--roneneldan--TinyStories-33M\snapshots\85c27af33c821254dc9b5a90618b4b10c57251d3\vocab.json
loading file merges.txt from cache at C:\Users\siddhanna.janai/.cache\huggingface\hub\models--roneneldan--TinyStories-33M\snapshots\85c27af33c821254dc9b5a90618b4b10c57251d3\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\siddhanna.janai/.cache\huggingface\hub\models--roneneldan--TinyStories-33M\snapshots\85c27af33c821254dc9b5a90618b4b10c57251d3\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\siddhanna.janai/.cache\huggingface\hub\models--roneneldan--TinyStories-33M\snapshots\85c27af33c821254dc9b5a90618b4b10c57251d3\tokenizer_config.json


In [65]:
# Define the tokenization function
def tokenize_function(row):
    merged = row["text"] + " " + row["target"]
    tokenized = tokenizer(merged, padding='max_length', truncation=True, max_length=100)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Apply the tokenization function on the DataFrame
tokenized_datasets = train.apply(tokenize_function, axis=1)

# Print the tokenized datasets
print(tokenized_datasets)


0     [input_ids, attention_mask, labels]
1     [input_ids, attention_mask, labels]
2     [input_ids, attention_mask, labels]
3     [input_ids, attention_mask, labels]
4     [input_ids, attention_mask, labels]
                     ...                 
75    [input_ids, attention_mask, labels]
76    [input_ids, attention_mask, labels]
77    [input_ids, attention_mask, labels]
78    [input_ids, attention_mask, labels]
79    [input_ids, attention_mask, labels]
Length: 80, dtype: object


In [41]:
# batch["labels"]

In [67]:
# Let's check out one prepared example
print(tokenizer.decode(tokenized_datasets[4]['input_ids']))

Character Name: Yama Race: Deva Class: God of Death Yama was the god of death. He was a powerful god and a just judge. He was a friend of the Pandavas and helped them in the Kurukshetra War.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [68]:
# Apply the tokenization function on the DataFrame
tokenized_datasets2 = test.apply(tokenize_function, axis=1)

In [69]:
tokenized_datasets2

0     [input_ids, attention_mask, labels]
1     [input_ids, attention_mask, labels]
2     [input_ids, attention_mask, labels]
3     [input_ids, attention_mask, labels]
4     [input_ids, attention_mask, labels]
5     [input_ids, attention_mask, labels]
6     [input_ids, attention_mask, labels]
7     [input_ids, attention_mask, labels]
8     [input_ids, attention_mask, labels]
9     [input_ids, attention_mask, labels]
10    [input_ids, attention_mask, labels]
11    [input_ids, attention_mask, labels]
12    [input_ids, attention_mask, labels]
13    [input_ids, attention_mask, labels]
14    [input_ids, attention_mask, labels]
15    [input_ids, attention_mask, labels]
16    [input_ids, attention_mask, labels]
17    [input_ids, attention_mask, labels]
18    [input_ids, attention_mask, labels]
19    [input_ids, attention_mask, labels]
dtype: object

In [70]:
# We will train a causal (autoregressive) language model from a pretrained checkpoint
model = AutoModelForCausalLM.from_pretrained(model_checkpoint);

loading configuration file config.json from cache at C:\Users\siddhanna.janai/.cache\huggingface\hub\models--roneneldan--TinyStories-33M\snapshots\85c27af33c821254dc9b5a90618b4b10c57251d3\config.json
Model config GPTNeoConfig {
  "_name_or_path": "roneneldan/TinyStories-33M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      2
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 16,
  "num_layers": 4,
  "resid_dropout": 0,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels":

In [72]:
# Start a new wandb run
run = wandb.init(project='dlai_lm_tuning', job_type="training", anonymous="allow",reinit=True)

#run = wandb.init(project=PROJECT, job_type="generation",reinit=True)

In [85]:
# Define training arguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-characters-backstories",
    report_to="wandb", # we need one line to track experiments in wandb
    num_train_epochs=4,
    logging_steps=1,
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    weight_decay=0.01,
    no_cuda=True, # force cpu use, will be renamed `use_cpu`
)

In [164]:
#pip install transformers[torch]

In [86]:
# We'll use HF Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets2,
)

In [87]:
# Let's train!
trainer.train()

***** Running training *****
  Num examples = 80
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 40
  Number of trainable parameters = 68514048
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'loss': 1.0918, 'learning_rate': 0.000975, 'epoch': 0.1}
{'loss': 7.0412, 'learning_rate': 0.00095, 'epoch': 0.2}
{'loss': 11.736, 'learning_rate': 0.000925, 'epoch': 0.3}
{'loss': 24.6903, 'learning_rate': 0.0009000000000000001, 'epoch': 0.4}
{'loss': 11.1245, 'learning_rate': 0.000875, 'epoch': 0.5}
{'loss': 6.0597, 'learning_rate': 0.00085, 'epoch': 0.6}
{'loss': 7.2999, 'learning_rate': 0.000825, 'epoch': 0.7}
{'loss': 4.8696, 'learning_rate': 0.0008, 'epoch': 0.8}
{'loss': 4.5734, 'learning_rate': 0.0007750000000000001, 'epoch': 0.9}


***** Running Evaluation *****
  Num examples = 20
  Batch size = 8


{'loss': 4.5412, 'learning_rate': 0.00075, 'epoch': 1.0}
{'eval_loss': 3.967233180999756, 'eval_runtime': 2.9962, 'eval_samples_per_second': 6.675, 'eval_steps_per_second': 1.001, 'epoch': 1.0}
{'loss': 3.0074, 'learning_rate': 0.000725, 'epoch': 1.1}
{'loss': 3.4309, 'learning_rate': 0.0007, 'epoch': 1.2}
{'loss': 3.2137, 'learning_rate': 0.000675, 'epoch': 1.3}
{'loss': 2.8295, 'learning_rate': 0.0006500000000000001, 'epoch': 1.4}
{'loss': 2.3952, 'learning_rate': 0.000625, 'epoch': 1.5}
{'loss': 2.8198, 'learning_rate': 0.0006, 'epoch': 1.6}
{'loss': 2.3197, 'learning_rate': 0.000575, 'epoch': 1.7}
{'loss': 2.9427, 'learning_rate': 0.00055, 'epoch': 1.8}
{'loss': 2.1566, 'learning_rate': 0.0005250000000000001, 'epoch': 1.9}


***** Running Evaluation *****
  Num examples = 20
  Batch size = 8


...

***** Running Evaluation *****
  Num examples = 20
  Batch size = 8


{'loss': 1.1281, 'learning_rate': 0.0, 'epoch': 4.0}




Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 2.0273594856262207, 'eval_runtime': 3.4726, 'eval_samples_per_second': 5.759, 'eval_steps_per_second': 0.864, 'epoch': 4.0}
{'train_runtime': 177.4108, 'train_samples_per_second': 1.804, 'train_steps_per_second': 0.225, 'train_loss': 3.5792510151863097, 'epoch': 4.0}


TrainOutput(global_step=40, training_loss=3.5792510151863097, metrics={'train_runtime': 177.4108, 'train_samples_per_second': 1.804, 'train_steps_per_second': 0.225, 'train_loss': 3.5792510151863097, 'epoch': 4.0})

In [92]:
# Start a new wandb run
run = wandb.init(project='dlai_lm_tuning', job_type="training", anonymous="allow",reinit=True)

#run = wandb.init(project=PROJECT, job_type="generation",reinit=True)

In [94]:
transformers.logging.set_verbosity_error() # suppress tokenizer warnings

prefix = "Generate Back story for the following information on Mahabharatha Character Name: "

prompts = [
    "Character Name: Arjuna Character Race: Human  Character Class: Prince ",
    "Character Name: Dhaumya Character Race: Rishi Character Class: Sage",
    "Character Name: Drona Character Race: Human Character Class: Guru and Worrier ",
    "Character Name: Krishna Character Race: Avatar Character Class: Politician",
]

table = wandb.Table(columns=["prompt", "generation"])

for prompt in prompts:
    input_ids = tokenizer.encode(prefix + prompt, return_tensors="pt")
    output = model.generate(input_ids, do_sample=True, max_new_tokens=50, top_p=0.3)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    table.add_data(prefix + prompt, output_text)

wandb.log({'tiny_generations': table})

wandb: WARNING Artifacts logged anonymously cannot be claimed and expire after 7 days.


In [95]:
wandb.finish()

In [2]:
#output_text